In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%pdb off

from pyCascade import probePost, physics, utils, probeReadWrite
from pyCascade.probeReadWrite import read_probes_file_switch
from filloutVentilationStats import *
from matplotlib import pyplot as plt
from matplotlib import cm, colors
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
import numpy as np
import scipy as sp
import os
from IPython.core.debugger import set_trace
import pandas as pd
import seaborn as sns
from cycler import cycler
import plotly.express as px
import plotly
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import statsmodels.api as sm
import warnings
import ast
from scipy.optimize import minimize
import seaborn as sns

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

plt.rcParams['figure.dpi'] = 140
im_scaling = .75
plt.rcParams['figure.figsize'] = [6.4 * im_scaling, 4.8 * im_scaling]

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')

# Setup

## Runs

In [2]:
multiRun_dir = f"{home_dir}/CHARLES/multiRuns/"
plotFolder = f"{multiRun_dir}"

## Read in results

In [3]:
def evalStringAsList(s):
    if isinstance(s, str) and s[0] == "[" and s[-1] == "]":
        return ast.literal_eval(s)
    else:
        return s

In [ ]:
runs = {
    247: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000], 'stops': [120000], 'delT': [0], 'SS': [True], "Fine": [False]},
    250: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000], 'stops': [120000], 'delT': [0], 'SS': [True], "Fine": [True]},
    254: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000], 'stops': [120000], 'delT': [0], 'SS': [True], "Fine": [False]},
    265: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000], 'stops': [120000], 'delT': [0], 'SS': [True], "Fine": [True]},

}
    
flowStatsMI, roomVentilationMI = readRunStats(runs, home_dir, scratch_dir, multiRun_dir = plotFolder, readABLFits = False, saveResults = False)

flowStatsMI = combine_stats(flowStatsMI, ["WS", "delT", "SS", "C", "A", "Fine"])
roomVentilationMI = combine_stats(roomVentilationMI,   ["WS", "delT", "SS", "C", "A", "Fine"])

In [ ]:
plotdf_all = roomVentilationMI[roomVentilationMI["houseType"] != "sl"].copy()
plotdf_all = plotdf_all[plotdf_all["SS"] == True]
plotdf_all["mean-mass_flux(S)-Norm"] /= -rho
plotdf_all["mean-mass_flux-Norm"] /= rho
plotdf_all["q-D-room-Norm"] /= rho

r_lim = [.59, .59, .59, .19]

fig, axs = plt.subplots(2, 4, subplot_kw={'projection': 'polar'}, figsize = (16, 10), dpi = 600, layout="tight")
colors = plt.cm.get_cmap("Set2").colors  # Use Matplotlib's Set2 colormap

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j in range(2):
        plotdf = plotdf_all.copy()
        plotdf = plotdf_all[plotdf_all["roomType"] == room]
        plotdf = plotdf[(plotdf["Fine"] == True) == j]
    
        x_var = "AofA"
        y_vars = ["mean-mass_flux(S)-Norm", "mean-mass_flux-Norm", "mean-sn_prod(abs(u))-Norm", "q-D-room-Norm"]

        # Create the polar plot
        ax = axs[j, i]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars
        plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True) #CHANGED TO MEDIAN
        angles = np.deg2rad(plotdfGrouped[x_var])
        radii = np.mean(plotdfGrouped[y_vars].values, axis = 1)
        ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth = 0.25) #CHANGED TO MEDIAN
        for k, y_var in enumerate(y_vars):
            width = np.pi/35 * 5 / 3
            offset = (len(y_vars)/2.5 - k)*width
            angles = np.deg2rad(plotdfGrouped[x_var]) + offset
            radii = plotdfGrouped[y_var].values / rho
            # color = "lightgrey"
            color = colors[k]
            ax.bar(angles, radii, width=width, color=color, edgecolor = 'white', linewidth = .5, alpha = 1)
        
        # Adding a legend for the custom error bars
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])
        if j == 0:
            ax.title.set_text(room)
        if i == 0:
            ax.set_ylabel(f"Fine: {bool(j)}", fontsize=16)

In [ ]:
plotdf_all = roomVentilationMI[roomVentilationMI["houseType"] != "sl"].copy()
plotdf_all = plotdf_all[plotdf_all["SS"] == True]
plotdf_all["mean-mass_flux(S)-Norm"] /= -rho
plotdf_all["mean-mass_flux-Norm"] /= rho
plotdf_all["q-D-room-Norm"] /= rho

r_lim = [.59, .59, .59, .19]

fig, axs = plt.subplots(2, 4, subplot_kw={'projection': 'polar'}, figsize = (16, 10), dpi = 600, layout="tight")
colors = plt.cm.get_cmap("Set2").colors  # Use Matplotlib's Set2 colormap

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j in range(2):
        plotdf = plotdf_all.copy()
        plotdf = plotdf_all[plotdf_all["roomType"] == room]
        plotdf = plotdf[(plotdf["Fine"] == True) == j]
    
        x_var = "AofA"
        y_var = "mean-sn_prod(abs(u))-Norm"

        # Create the polar plot
        ax = axs[j, i]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars
        plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True) #CHANGED TO MEDIAN
        angles = np.deg2rad(plotdfGrouped[x_var])
        radii = plotdfGrouped[y_var].values
        ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth = 0.25) #CHANGED TO MEDIAN
        h_order = ["0--2", "0--1", "0-0", "0-1", "0-2"]
        for k, h in enumerate(h_order):
            width = np.pi/35
            offset = (len(h_order)/2.5 - k)*width

            plotdfHouse = plotdf[plotdf["houseType"] == h]
            plotdfGrouped = plotdfHouse.groupby(x_var, as_index=False).median(numeric_only=True) 
            angles = np.deg2rad(plotdfGrouped[x_var]) + offset
            radii = plotdfGrouped[y_var].values
            # color = "lightgrey"
            color = colors[k]
            ax.bar(angles, radii, width=width, color=color, edgecolor = 'white', linewidth = .5, alpha = 1)
        
        # Adding a legend for the custom error bars
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])
        if j == 0:
            ax.title.set_text(room)
        if i == 0:
            ax.set_ylabel(f"Fine: {bool(j)}", fontsize=16)

In [ ]:
dfDiff = roomVentilationMI.groupby(
    ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"], as_index=False
).apply(
    lambda group: group.select_dtypes(include='number').iloc[0] - group.select_dtypes(include='number').iloc[1]
)

def rms(x):
    return np.sqrt(np.mean(x**2))

rms(dfDiff["mean-sn_prod(abs(u))-Norm"])

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
# plotdf = plotdf.sample(frac=1)
# plotdf = plotdf.sort_values(["Params"])
plotdf = plotdf.sort_values(["Params", "houseType"])

categorizeBy = ["C", "roomType"]
categoryValues = [plotdf[category].unique() for category in categorizeBy]
splitAlong = ["Fine"]
splitInd = [0, 1]
qoi = "q-D-room-Norm"
figures = []
for i, v0 in enumerate(categoryValues[0]):
    for j, v1 in enumerate(categoryValues[1]):
        plotdfSplit = plotdf[((plotdf[categorizeBy[0]] == v0) & (plotdf[categorizeBy[1]] == v1))]
        dfComp, values = getComparativeDf(plotdfSplit, splitAlong)
        color = dfComp[('A', *values[0])]
        color = color.astype(str)
        fig = px.scatter(x = dfComp[qoi].iloc[:,splitInd[0]], y = dfComp[qoi].iloc[:,splitInd[1]], 
            color = color, symbol = indMulti(dfComp.index, ['WS']),  hover_data = indMulti(dfComp.index, ['C', 'roomType', 'AofA']),
            symbol_sequence=['circle', 'triangle-up'])
        figures.append(fig)

num_rows = len(categoryValues[0])
num_cols = len(categoryValues[1])

titleText = []
for CValue in categoryValues[1]:
    text = f"{categorizeBy[1]} = "
    if isinstance(CValue, float):
        text = f"{text}{np.round(CValue, 2)}"
    else:
        text = f"{text}{CValue}"
    titleText.append(text)

fig = make_subplots(rows=num_rows, cols=num_cols, vertical_spacing=0.2)#, subplot_titles=titleText)

for i, figure in enumerate(figures):
    for trace in range(len(figure["data"])):
        row = int(i / num_cols) + 1  # Calculate row index
        col = (i % num_cols) + 1  # Calculate column index
        # print(row, col)
        trace = figure["data"][trace]
        if i>0:
            trace.showlegend = False
        fig.add_trace(trace, row=row, col=col)
        # fig.update_annotations(y=1.05 - (row-1)/1.65, selector={'text':room})

# fig.update_polars(radialaxis_dtick=.1, radialaxis_range=[0,.4])
# fig.update_polars(radialaxis_nticks=5, radialaxis_range=[0,.4])
fig.update_layout(width = 1000, height = 800, title_text = f"{splitAlong[0]} {values[splitInd[0]][0]} vs {values[splitInd[1]][0]}")
fig.update_coloraxes(colorscale = "mrybm", colorbar_orientation = "h", colorbar_y = -0.2, cmin = 0, cmax = 360)

max_value = np.max(plotdf[qoi]) * 1.2
fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value, line=dict(color="Black"), row="all", col="all")

fig.add_shape(type="line", x0=0, y0=0, x1=max_value, y1=max_value * 0.75, 
              line=dict(color="Black", dash="dash"), row="all", col="all")
fig.add_shape(type="line", x0=0, y0=0, x1=max_value * 0.75, y1=max_value, 
              line=dict(color="Black", dash="dash"), row="all", col="all")


# Update x-axis and y-axis properties for each subplot
for i, v0 in enumerate(categoryValues[0]):
    yLabelText = f"{categorizeBy[0]} = {v0}"
    fig.update_yaxes(title_text=yLabelText, row=i+1, col=1)  # Add y-axis label

for j, v1 in enumerate(categoryValues[1]):
    xLabelText = f"{categorizeBy[1]} = {v1}"
    fig.update_xaxes(title_text=xLabelText, row=num_rows, col=j+1)  # Add x-axis label

fig.show()